# Chapitre 1

L'idée avec ce Chapitre c'est de pouvoir explorer le dataset dans sa globalité, essayer d'étudier les différentes corrélations entre les données et de pouvoir trouver des premiers insights ou patterns qui nous serviront par la suite pour le traitement de ces données

1. Data Exploration
2. Data Interpretation
3. Data Quality Problems
4. Data Cleaning
5. Text Processing
6. Data Transformation
7. Manifold Learning
8. Visualisation & Conclusions

In [2]:
import pandas as pd 
import polars as pl

In [3]:
"""Chargement du dataset en mode snapshot 
La data choisie est le 20-10-2025, certaines données peuvent varier mais pour l'instant on part du principe que ce dataset est représentatif
pour les travaux à venir"""

#Chargement du snapshot va se faire avec pandas mais à terme on va migrer sur du polars ...  ,
# plus rapide et avec du multithreading mais on va devoir faire du casting vers pandas pour les librairies comme scikit ou seaborn ... 

SNAPSHOT = "snapshot_dataset_20_10_2025.csv"
CHEMIN_FICHIER = f"./data/{SNAPSHOT}"  

DATASET = pd.read_csv(CHEMIN_FICHIER)
print(DATASET.shape)
print(DATASET.dtypes)

(28180, 18)
Timestamp                                                                                                                                                                                                                                object
How old are you?                                                                                                                                                                                                                         object
What industry do you work in?                                                                                                                                                                                                            object
Job title                                                                                                                                                                                                                                object
If your job title needs addi

Affichage des informations générales sur le dataset

In [4]:
print(DATASET.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28180 entries, 0 to 28179
Data columns (total 18 columns):
 #   Column                                                                                                                                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                --------------  -----  
 0   Timestamp                                                                                                                                                                                                                             28180 non-null  object 
 1   How old are you?                                                                                             

*** Remarque *** : Ici on remarque que nous avons une colonne numérique au premier abord qui est : How much additional monetary compensation do you have , donc les bonus etc est la seule colonne numérique directement, cependant le annual salary n'est pas en float ... faudra faire le casting.

Cette colonne inclut pas le remboursement du titre de transport, tickets resto etc ... 


## Renommage des colonnes
Pour commencer la visualisation et l'exploration plus détaillée, l'idée c'est de renommer les colonnes avant pour un meilleur confort visuel

In [5]:
df = DATASET
#Supposition : TODO autres_devises devrait être probablement fusionné avec devise, car c'est juste un formulaire et le choix se portait sur les plus connues
# TODO : concaténer état et pays si c'est USA , ville pas forcèment.... 
nouvelles_colonnes = ["timestamp","tranche_age","industrie_travail","intitule_poste","intitule_poste_contexte_supp","salaire_annuel","bonus",
"devise","autre_devise","contexte_revenus","pays_travail","etat_travail","ville_travail","experience_generale","experience_domaine",
"diplome","genre","race"]

def renommer_colonnes(df:pd.DataFrame, nouvelles_colonnes : list) -> pd.DataFrame : 
    dictionnaire_colonnes = {}
    for anc_col , nouv_col in zip(df.columns,nouvelles_colonnes):
        dictionnaire_colonnes[anc_col]=nouv_col
    df_nouveau = df.rename(columns=dictionnaire_colonnes)
    return df_nouveau

df_colonnes = renommer_colonnes(df,nouvelles_colonnes)
print(df_colonnes.head())


            timestamp tranche_age              industrie_travail  \
0  4/27/2021 11:02:10       25-34   Education (Higher Education)   
1  4/27/2021 11:02:22       25-34              Computing or Tech   
2  4/27/2021 11:02:38       25-34  Accounting, Banking & Finance   
3  4/27/2021 11:02:41       25-34                     Nonprofits   
4  4/27/2021 11:02:42       25-34  Accounting, Banking & Finance   

                             intitule_poste intitule_poste_contexte_supp  \
0        Research and Instruction Librarian                          NaN   
1  Change & Internal Communications Manager                          NaN   
2                      Marketing Specialist                          NaN   
3                           Program Manager                          NaN   
4                        Accounting Manager                          NaN   

  salaire_annuel   bonus devise autre_devise contexte_revenus    pays_travail  \
0         55,000     0.0    USD          NaN         

In [6]:
def comparaison_colonnes(df:pd.DataFrame,col1:str,col2:str):
    """Prendre deux colonnes et essayer de comparer les résultats """
    for i , row in df.iterrows():
        if str(row[col2])!="nan":
            print(f"indice {i} : {row[col1]} {row[col2]}\n\n")


def get_pourcentage_valeurs_nulles(df: pd.DataFrame , colonne : str ) -> float : 
    """prend une colonne en particulier et retourne le pourcentage de valeurs nulles contenant"""
    return (df[colonne].isnull().sum()/df.shape[0])*100


# comparaison_colonnes(df_colonnes,"devise","autre_devise")

seuil_valeurs_nulles = 10 #10% sont nulles 
for colonne in df_colonnes.columns : 
    if get_pourcentage_valeurs_nulles(df_colonnes,colonne) >= 10 : 
        print(f"colonne {colonne} est nulle à {get_pourcentage_valeurs_nulles(df_colonnes,str(colonne))}")



colonne intitule_poste_contexte_supp est nulle à 74.15188076650107
colonne bonus est nulle à 26.103619588360537
colonne autre_devise est nulle à 99.22995031937545
colonne contexte_revenus est nulle à 89.17672107877928
colonne etat_travail est nulle à 17.94180269694819


Observations : 

. colonne autre devise vide à 99% => envisager factorisation

. colonne devise pas vide mais cas où nous avons des others, cas autres devises obligatoire

. colonne bonus 26% nulles mais peut être pertinente ...

. etat_travail nulle à 18% indique que la plupart des travailleurs viennent des US .. à voir si c'est vraiment pertinent ou si on factoriserait pas selon le besoin

. contexte revenus nulle à 90% mais à garder certains pouvant justifier leurs revenus avec ça

. intitule poste contexte nulle à 75% .. à garder pour l'instant




In [7]:
comparaison_colonnes(df_colonnes,"devise","autre_devise")

indice 434 : Other INR


indice 603 : Other Peso Argentino


indice 752 : USD $76,302.34


indice 766 : USD My bonus is based on performance up to 10% of salary


indice 776 : USD I work for an online state university, managing admissions data. Not direct tech support. 


indice 1001 : USD 0


indice 1311 : Other MYR


indice 1840 : Other CHF


indice 1915 : USD KWD


indice 1924 : Other NOK


indice 2048 : USD Na 


indice 2054 : Other NOK


indice 2473 : Other USD


indice 2639 : Other BR$


indice 2980 : Other SEK


indice 3124 : USD Base plus Commission 


indice 3145 : CAD canadian


indice 3162 : Other Dkk


indice 3261 : Other EUR


indice 3605 : USD COP


indice 3937 : Other NOK


indice 4264 : Other TTD


indice 4499 : Other Indian rupees


indice 4780 : Other BRL (R$)


indice 4971 : Other Mexican pesos


indice 5146 : Other CZK


indice 5306 : Other CZK


indice 5396 : Other GBP


indice 5729 : Other NOK


indice 5871 : Other GBP


indice 6128 : Other DKK


indice 6765 : Oth

In [8]:
import matplotlib.pyplot as plt
from typing import Literal , List , Optional
import seaborn as sea

In [10]:
def visualisation_distribution_donnees(df : pd.DataFrame , colonne : str , type_graphique:Literal["bar","pie","hist"]) : 
    """visualiser selon le type de graphique et la colonne qu'on choisit"""
    figsize = (10,8)
    plt.figure(figsize=figsize)
    if type_graphique == "bar" : 
        sea.barplot(x=df[colonne].value_counts().index,y=df[colonne].value_counts().values)
        plt.title(f'Distribution de {colonne}')
        plt.xlabel(f'{colonne}')
        plt.ylabel('Count')
        plt.show()  
    if type_graphique == "pie":
        plt.pie(df[colonne].value_counts().values, labels=df[colonne].value_counts().index, autopct='%1.1f%%', startangle=90,labeldistance=1.2)
        plt.title(f'Distribution de {colonne}')
        plt.axis('equal') 
        plt.show()
    if type_graphique == "hist":
        sea.histplot(data=df,x=colonne,bins=300,log_scale=False)
        plt.ticklabel_format(style='plain', axis='x')  #désactive le exposant6 et affiche le chiffre dans son entièreté.
        plt.show()


def verifier_colonne_numerique(df:pd.DataFrame,colonne:str) -> None : 
    """Cette fonction viseprincipalement à faire en sorte de vérifier certaines colonnes numériques parfois problématiques"""
    print(df[colonne].describe())  
    print("Max:", df[colonne].max())
    print("Min:", df[colonne].min())


# df_colonnes["bonus"] = df_colonnes["bonus"].dropna()
moyenne = df_colonnes["bonus"].mean()
ecart_type = df_colonnes["bonus"].std()
print(df_colonnes['bonus'].dropna().sort_values())#on affiche le mode qui pour l'instant est à 0 .. ici 
# print("moyenne", moyenne ,"ecart_type",ecart_type,"mode")

# verifier_colonne_numerique(df_colonnes , "bonus")


0              0.0
16126          0.0
16128          0.0
16129          0.0
7170           0.0
           ...    
18322    1400000.0
18279    1400000.0
26935    1500000.0
26980    2000000.0
28100    2000000.0
Name: bonus, Length: 20823, dtype: float64


Observation : on remarque que la valeur max en temps normal pour la colonne bonus la valeur max est à 12 millions, 
mà où la moyenne est beaucoup plus basse, dans ce cas on va décider de garder que les valeurs 3 sigmas au délà de la moyenne au délà on affiche la colonne pour investiguer 

Il s'agit d'un problème de la monnaie avec lesquels les gens sont payés ... faudra faire un comparatif sur la distribution de données apparatenant à la même catégorie devise ...